# Data Filtering

#### Objective 🥅

Data in the real world is far from perfect. This notebook exposes the workshop participants to a workflow for filtering sensor data. 

### Scenario Epsilon

In this notebook, we will use a combined stormwater network with eleven controllable assets as our case study. The orange nodes in the image below represent these controllable assets.

![epsilon](./data/epsilon.png)

**Problem Statement**

We have recorded measured data from all eleven controllable locations during consecutive storm events, and we want to estimate the storage utilization in the network.  

### Exercise 1
Load, visuvalize, and figure out what is hiding in the data 🔍

In [ ]:
# Note: We will be relying on pandas a lot to help us deal with timeseries data
# if you haven't used pandas before, we cannot recommend it enough! 
# We ❤️ 🐼
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# RC parms for pretty plots 💁🏽
plt.rcParams.update({'font.size': 14})
plt.style.use('seaborn-v0_8-whitegrid')
plt.style.use('seaborn-v0_8-dark-palette')

In [ ]:
# load the measured data
measured_data = pd.read_csv("./data/measured_data_epsilon.csv")

In [ ]:
measured_data

Each column represents the depth measurement data in **ft** in the storage asset represented by the orange nodes in the above figure. The column <code>Unnamed: 0</code> represents the time of the measurement.

In [ ]:
# let us set the timestamp as the index of the index
# index will help us better query depth data
measured_data = measured_data.set_index("Unnamed: 0")
# convert index to datetime object
measured_data.index = pd.to_datetime(measured_data.index)

In [ ]:
plt.figure(figsize=(10, 5), dpi=125)
plt.plot(measured_data['004'])
plt.title("Depth in 004")
plt.ylabel("Depth (ft)")
plt.xlabel("Timestep")
# Rotate axis so that they are readable
ax = plt.gca()
ax.tick_params(axis='x', labelrotation = 45)
plt.show()

Something is going on. Clearly, depth cannot be negative. Furthermore, it cannot rapidly go from 3 to 20. So we need to remove anomalies. But let us zoom into it to see if there is something else going on

In [ ]:
# Zoom and enhance
# Let us take a look at the diurnal patters to see if need to handle anything else
depth_4 = measured_data['004'].loc[pd.Timestamp("2017-01-12"):pd.Timestamp("2017-01-14")]

plt.figure(figsize=(20, 5), dpi=100)
plt.plot(depth_4)
plt.ylabel("Depth (ft)")
plt.xlabel("Timestep")
# Rotate axis so that they are readable
ax = plt.gca()
ax.tick_params(axis='x', labelrotation = 45)
plt.show()

That does not look like a diurnal pattern. However, if you squint, it does look like a signal is hiding amongst the noise. Furthermore, it seems there are some gaps in the data as well. Let us take a look to see how these gaps are represented. 

In [ ]:
measured_data['004'].loc[pd.Timestamp("2017-01-06"):pd.Timestamp("2017-01-07")]

Looks like there are some NaNs hiding in the data as well. There are a lot types of NaNs in Python 😭, pandas has <code>dronna()</code> function which will solve all our problems.

**Summary**
1. There are outliers in the depth data that have be removed.
2. There is measurement noise in the data.
3. There are NaNs in the data.

🏋🏽 **Exercise 1.1**

Explore the data to see if there is anything else hiding?

😉 Hint 1 -> Take a look at site 006 

😉 Hint 2 -> Gradient can be computed using <code>measured_data['004'].diff()<code>

In [ ]:
# Querying data for for a particular site
measured_data['006']

In [ ]:
# Estimate gradient
measured_data['004'].diff()

In [1]:
# Use gradients to figure the hiding anamoly
# < Your amesome code goes here >

### Exercise 2

Based on the issues identified in Exercise-1, clean the data! 

In [ ]:
depth_4 = measured_data['004']

In [ ]:
# Let us remove NaN values first
depth_4 = depth_4.dropna()
# Let us clear off the negative values next
depth_4 = depth_4[depth_4 > 0.0]
# Remove the flat lines
# 1. gradient of 0.0 means flatline
gradient = depth_4.diff()
# 2. identify the timesteps when gradients are 0
index_flatlines = depth_4[depth_4.diff() == 0.0].index
# 3. remove these values
depth_4 = depth_4.drop(index_flatlines)

In [ ]:
plt.figure(figsize=(10, 5), dpi=125)
plt.plot(depth_4)
plt.title("Depth in 004")
plt.ylabel("Depth (ft)")
plt.xlabel("Timestep")
# Rotate axis so that they are readable
ax = plt.gca()
ax.tick_params(axis='x', labelrotation = 45)
plt.show()

Now to the fun stuff! Let's remove the anomalies from the data. Thankfully, they look obvious; let us try something simple.

In [ ]:
# 4. remove the anamolies
mean = depth_4.mean()
std = depth_4.std()
# Let remove everything that exceeds 95% CI
upper_limit = mean + 2 * std

In [ ]:
upper_limit

In [ ]:
depth_4[depth_4 > upper_limit]

In [ ]:
up_limit_excess = depth_4[depth_4 > upper_limit].index

In [ ]:
depth_4 = depth_4.drop(up_limit_excess)

In [ ]:
plt.figure(figsize=(10, 5), dpi=125)
plt.plot(depth_4)
plt.title("Depth in 004")
plt.ylabel("Depth (ft)")
plt.xlabel("Timestep")
# Rotate axis so that they are readable
ax = plt.gca()
ax.tick_params(axis='x', labelrotation = 45)
plt.show()

🏋🏽 **Exercise 2.1**

Using the above code, clean the data for rest of the sites

In [ ]:
cleaned_depth_data = pd.DataFrame()

for site in measured_data.columns:
    # Isolate the data for a site 
    depth_site = measured_data[site]
    
    # Let us remove NaN values first

    
    # Let us clear off the negative values next

    
    # Remove the flat lines
    # 1. gradient of 0.0 means flatline

    # 2. identify the timesteps when gradients are 0

    # 3. remove these values

    
    
    # 4. remove the anamolies

    # Let remove everything that exceeds 95% CI

    # Note: concat function helps stack the cleaned data into columns
    # We assume depth_site is the name of the cleaned data 
    # This is equivalent to the depth_4 above.
    cleaned_depth_data = pd.concat([cleaned_depth_data, depth_site], axis=1)

### Exercise 3

Now that we have cleaned the apparent stuff, how do we clean the measurement noise that is obfuscating our data? Let us take some help from my favorite Frenchman, Joseph Fourier

![Fourier](https://upload.wikimedia.org/wikipedia/commons/thumb/d/df/Fourier2_-_restoration1.jpg/440px-Fourier2_-_restoration1.jpg)

We will use a 200 year old method to help us extract our signal.
[Denoising Data using FFT](https://www.youtube.com/watch?v=s2K1JfNR7Sc) provides a really good overview of the fourier filtering approach.

In [ ]:
def fourier_filtering(signal, t, dt, psd_threshold):
    n = len(t)
    # Compute the FFT
    fhat = np.fft.fft(signal,n)
    # Power spectrum (power per freq)
    PSD = fhat * np.conj(fhat) / n
     # Create x-axis of frequencies in Hz
    freq = (1/(dt*n)) * np.arange(n)
    # Only plot the first half of freqs
    L = np.arange(1, np.floor(n/2),dtype='int')
    # Find all freqs with large power
    indices = PSD > psd_threshold
    # Zero out all others
    PSDclean = PSD * indices
    # Zero out small Fourier coeffs. in Y
    fhat = indices * fhat
    # Inverse FFT for filtered time signal
    ffilt = np.fft.ifft(fhat)
    return freq, PSD, ffilt, L, PSDclean

In [ ]:
def filter_columns(data: pd.DataFrame, psd_threshold: float, plot: bool):
    # convert datetime index to seconds starting from 0
    column_name = data.columns[0]
    depth_site = data[[column_name]]
    depth_site = depth_site.resample("15min").mean().interpolate()
    depth_site['time'] = depth_site.index.values
    depth_site['time'] = (depth_site['time'] - depth_site['time'][0]).apply(lambda x: x.total_seconds())
    
    # Note: 1/900 as each timestep is 15min = 900s econds
    freq, PSD, ffilt, L, PSDclean = fourier_filtering(signal=depth_site[column_name].values,
                                                      t=depth_site['time'].values,
                                                      dt=1/900.0,
                                                      psd_threshold=psd_threshold)
    
    # Ignore the imaginary numbers, they are just in my head
    ffilt = ffilt.real
    
    if plot:
        plt.figure(figsize=(20, 10), dpi=125)
        plt.subplot(2, 1, 1)
        plt.plot(depth_site.index, depth_site[column_name].values, label='Noisy', alpha=0.5)
        plt.plot(depth_site.index, ffilt,color='b',label='Filtered', linewidth=2.0)
        plt.ylabel("Depth (ft)")
        plt.xlabel("Time")
        plt.legend()

        plt.subplot(2, 1, 2)
        plt.plot(freq[L],PSD[L],label='Noisy', alpha=0.9)
        plt.plot(freq[L],PSDclean[L],label='Filtered', linewidth=2.0)
        plt.xlim(freq[L[0]], 20)
        plt.xlabel("Frequency (Hz)")
        plt.ylabel("PSD")

        plt.legend()
        plt.suptitle(f"{column_name} Filtered")
        plt.show()
    else:
        return pd.DataFrame(data={column_name:ffilt}, index=depth_site.index)

In [ ]:
filter_columns(data=depth_4.to_frame(), psd_threshold=1.0, plot=True)

In [ ]:
# By setting the plot to false we can get the cleaned data
filter_columns(data=depth_4.to_frame(), psd_threshold=1.0, plot=False)

🏋🏽 **Exercise 3.1**

Using the above code generate clean data. There isn't one correct answer. You, as an engineer, will have to make a call on what is good enough for our goal of estimating the degree of utilization of the storage assets.

🧐 : Do we have to identify a new filtering threshold (<code>psd_threshold</code>) for each site, or can we use the same threshold for all the sensors?
    
🤔 : Can we use fourier filtering and avoid writing code to deal with flatlines and anomalies?
    
Note: Make sure you completed Exercise 2.1 and are using the <code>cleaned_depth_data</code> for this exercise.

In [ ]:
denoised_data = pd.DataFrame()

# TODO: figure out the right threshold for filtering data
# This is the 🎚️ knob you would want to dial 
psd_threshold_site = 0.0

for site in cleaned_depth_data.columns:
    # Isolate the data for a site 
    depth_site = cleaned_depth_data[[site]]
    
    # FFT!! 🪄
    depth_site = filter_columns(data=depth_site, psd_threshold=psd_threshold_site, plot=False)

    # concat data into a common DataFrame
    denoised_data = pd.concat([denoised_data, depth_site], axis=1)

🏋🏽 **Exercise 3.2**

Once we have the clean data, now let us compute the average utilization.

In [ ]:
average_depth = denoised_data.mean(axis=0).to_dict()

In [ ]:
average_depth

In [ ]:
max_depth = {"004": 14.7, 
             "006": 9,
             "011": 14,
             "022": 15.5,
             "027": 15.5,
             "030": 15.5,
             "033": 15.5,
             "039": 12.25,
             "044": 15.5,
             "050": 10.5,
             "060": 11.5
            }
utilization = {}

for site in average_depth.keys():
    utilization[site] = average_depth[site]/max_depth[site]

Based on the above data, make a decision on the available capacity in the system.

🤨: Is there any capacity in the system that we can levearge to improve the operation of the stormwater network?